## 1) Importing the packages

We will import [TensorFlow](https://www.tensorflow.org) and our beloved [Keras](https://www.tensorflow.org/guide/keras). Also, we import other modules which help in defining model layers.






In [0]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
import pandas as pd 


## 2) Preprocessing the data

### B) Reading the data from the files

We parse each of the `.yaml` files.

*   Concatenate two or more sentences if the answer has two or more of them.
*   Remove unwanted data types which are produced while parsing the data.
*   Append `<START>` and `<END>` to all the `answers`.
*   Create a `Tokenizer` and load the whole vocabulary ( `questions` + `answers` ) into it.





In [0]:

from tensorflow.keras import preprocessing , utils
import os,re
import yaml

questions = list()
answers = list()


stream = open('conversations.yml', 'rb')
docs = yaml.safe_load(stream)
conversations = docs['conversations']

questions_test = []
answers_test = []
num_lists = 0

for con in conversations:
  num_lists = num_lists + 1  

for i in range(0,num_lists):
  for x in range(len(conversations[i])):
    if len(conversations[i]) % 2 == 0:
      if x % 2 == 0:
        questions.append(conversations[i][x])
      else:
        answers.append('<START>' + conversations[i][x] + "<END>" )

FileNotFoundError: ignored

In [0]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
word2idx = tokenizer.word_index
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))



VOCAB SIZE : 242



### C) Preparing data for Seq2Seq model

Our model requires three arrays namely `encoder_input_data`, `decoder_input_data` and `decoder_output_data`.

For `encoder_input_data` :
* Tokenize the `questions`. Pad them to their maximum length.

For `decoder_input_data` :
* Tokenize the `answers`. Pad them to their maximum length.

For `decoder_output_data` :

* Tokenize the `answers`. Remove the first element from all the `tokenized_answers`. This is the `<START>` element which we added earlier.



In [0]:

MAX_NUM_WORDS = 1000
print('Loading word vectors...')
word2vec = {}
with open(os.path.join('cbow_s100.txt'),mode='r', encoding='utf-8-sig') as f:
  # is just a space-separated text file in the format:cbow_s100.txt'),mode='r', encoding='utf-8-sig') as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
  for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:])
    word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))

print('Filling pre-trained embeddings...')
num_words = min(MAX_NUM_WORDS, VOCAB_SIZE)
embedding_matrix = np.zeros((num_words,100))
for word, i in word2idx.items():
  if i < MAX_NUM_WORDS:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
      # words not found in embedding index will be all zeros.
      embedding_matrix[i] = embedding_vector


# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )


Loading word vectors...
Found 929595 word vectors.
Filling pre-trained embeddings...
(47, 26) 26
(47, 19) 19
(47, 19, 242)


## 3) Defining the Encoder-Decoder model
The model will have Embedding, LSTM and Dense layers. The basic configuration is as follows.


*   2 Input Layers : One for `encoder_input_data` and another for `decoder_input_data`.
*   Embedding layer : For converting token vectors to fix sized dense vectors. **( Note :  Don't forget the `mask_zero=True` argument here )**
*   LSTM layer : Provide access to Long-Short Term cells.

Working : 

1.   The `encoder_input_data` comes in the Embedding layer (  `encoder_embedding` ). 
2.   The output of the Embedding layer goes to the LSTM cell which produces 2 state vectors ( `h` and `c` which are `encoder_states` )
3.   These states are set in the LSTM cell of the decoder.
4.   The decoder_input_data comes in through the Embedding layer.
5.   The Embeddings goes in LSTM cell ( which had the states ) to produce sequences.



<center><img style="float: center;" src="https://cdn-images-1.medium.com/max/1600/1*bnRvZDDapHF8Gk8soACtCQ.gif"></center>


Image credits to [Hackernoon](https://hackernoon.com/tutorial-3-what-is-seq2seq-for-text-summarization-and-why-68ebaa644db0).










In [0]:

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 100 , mask_zero=True,weights = [embedding_matrix],trainable=False ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 100 , mask_zero=True,weights = [embedding_matrix],trainable = False ) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy',metrics=['acc'])

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    24200       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    24200       input_2[0][0]                    
______________________________________________________________________________________________

## 4) Training the model
We train the model for a number of epochs with `RMSprop` optimizer and `categorical_crossentropy` loss function.

In [0]:

model.fit([encoder_input_data , decoder_input_data], decoder_output_data,epochs=100,validation_split=0.2,batch_size = 50 ) 


Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 2.1905 - acc: 0.6046 - val_loss: 1.8868 - val_acc: 0.7105
Epoch 2/100
1/1 [==============================] - 0s 34ms/step - loss: 2.1520 - acc: 0.6728 - val_loss: 1.8590 - val_acc: 0.0526
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 2.1066 - acc: 0.0541 - val_loss: 1.7896 - val_acc: 0.0526
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.9813 - acc: 0.0526 - val_loss: 1.6248 - val_acc: 0.0526
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7996 - acc: 0.0526 - val_loss: 1.6251 - val_acc: 0.0947
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6874 - acc: 0.0953 - val_loss: 1.5894 - val_acc: 0.0684
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.6209 - acc: 0.0683 - val_loss: 1.6128 - val_acc: 0.1053
Epoch 8/100
1/1 [==============================] - 0s 38ms/step - loss: 1.5798 - acc: 0.1067

## 5) Defining inference models
We create inference models which help in predicting answers.

**Encoder inference model** : Takes the question as input and outputs LSTM states ( `h` and `c` ).

**Decoder inference model** : Takes in 2 inputs, one are the LSTM states ( Output of encoder model ), second are the answer input seqeunces ( ones not having the `<start>` tag ). It will output the answers for the question which we fed to the encoder model and its state values.

In [0]:

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


## 6) Talking with Chatbot

First, we define a method `str_to_tokens` which converts `str` questions to Integer tokens with padding.


In [0]:

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')





1.   First, we take a question as input and predict the state values using `enc_model`.
2.   We set the state values in the decoder's LSTM.
3.   Then, we generate a sequence which contains the `<start>` element.
4.   We input this sequence in the `dec_model`.
5.   We replace the `<start>` element with the element which was predicted by the `dec_model` and update the state values.
6.   We carry out the above steps iteratively till we hit the `<end>` tag or the maximum answer length.







In [0]:

enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation)


In [0]:
question = pd.Series(questions)
answer = pd.Series(answers)
quest2ans = pd.DataFrame({"questions":question,"answers":answer})
quest2ans

,questions,answers
0,Como vai você?,"<START>Eu estou bem, e você?<END>"
1,Eu também estou bem.,<START>Isso é bom.<END>
2,Ouviu as notícias?,<START>Que boa notícia?<END>
3,Qual é o seu livro favorito?,<START>Eu não sei ler.<END>
4,"Então, qual é a sua cor favorita?",<START>Azul<END>
5,O bolo é uma mentira.,"<START>Não, não é. O bolo é delicioso.<END>"
6,O que mais é delicioso?,<START>Nenhuma coisa<END>
7,Ou algo,<START>Fale-me sobre você.<END>
8,O que você quer saber?,<START>Você é um robô?<END>
9,Sim eu sou.,<START>Como é?<END>
